### The problem
Given a vector $\mathbf{b}$ and a Hermitian matrix $A$ the goal is to find a vector $\mathbf{x}$ satisfying the equation
$$
A\mathbf{x}=\mathbf{b}
$$
The classical and quantum algorithms to solve this problem in Qiskit can be found in the `linear_solvers` package within `qiskit.algorithms`.

The quantum algorithms can find a solution exponentially faster in the size of the system than their classical counterparts (i.e. logarithmic complexity instead of polynomial). However the cost for this exponential speedup is that we do not obtain the full solution vector, since reading the entries would already take linear time.

### Workflow
The interface for all algorithms to solve the linear system problem is `LinearSolver`. The problem to be solved is only specified when the `solve()` method is called:
```python
LinearSolver(...).solve(matrix, vector)
```

The simplest implementation would be:

In [1]:
import numpy as np
from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver
from qiskit.algorithms.linear_solvers.hhl import HHL

matrix = np.array([[1 / 2, 1 / 6, 0, 0], [1 / 6, 1 / 2, 1 / 6, 0], [0, 1 / 6, 1 / 2, 1 / 6], [0, 0, 1 / 6, 1 / 2]])
vector = np.array([1.0, -2.1, 3.2, -4.3])

quantum_solution = HHL().solve(matrix, vector)
classical_solution = NumPyLinearSolver().solve(matrix, vector / np.linalg.norm(vector))

C:\Users\AlmudenaCarreraVazqu\Contribute\hhlrefactor\qiskit-terra\qiskit\providers\basicaer\statevector_simulator.py:144: UserWarning: Option backend_options is not used by this backend
  return super().run(qobj, backend_options=backend_options)


`solve()` returns a `LinearSolverResult` object, which contains the following properties:
- `state`: either the circuit that prepares the solution or the solution as a vector
- `euclidean_norm`: the euclidean norm if the algorithm knows how to calculate it
- `observable`: the (list of) calculated observable(s)
- `circuit_results`: the results from the circuits

In [2]:
print('quantum euclidean norm:', quantum_solution.euclidean_norm)
print('classical euclidean norm:', classical_solution.euclidean_norm)
print('quantum state:')
print(quantum_solution.state)
print('classical state:', classical_solution.state)

quantum euclidean norm: 3.986685550278159
classical euclidean norm: 3.98635634469049
quantum state:
      ┌────────────┐┌──────┐        ┌─────────┐
q0_0: ┤0           ├┤3     ├────────┤3        ├
      │  circuit-7 ││      │        │         │
q0_1: ┤1           ├┤4     ├────────┤4        ├
      └────────────┘│      │┌──────┐│         │
q1_0: ──────────────┤0 QPE ├┤2     ├┤0 QPE_dg ├
                    │      ││      ││         │
q1_1: ──────────────┤1     ├┤1     ├┤1        ├
                    │      ││  1/x ││         │
q1_2: ──────────────┤2     ├┤0     ├┤2        ├
                    └──────┘│      │└─────────┘
q2_0: ──────────────────────┤3     ├───────────
                            └──────┘           
classical state: [ 0.96526675 -1.86892072  2.48504844 -2.30021012]


The `solve()` method accepts up to five arguments: 
```python
def solve(self, matrix: Union[np.ndarray, QuantumCircuit],
          vector: Union[np.ndarray, QuantumCircuit],
          observable: Optional[Union[LinearSystemObservable, BaseOperator,
                                     List[BaseOperator]]] = None,
          post_rotation: Optional[Union[QuantumCircuit, List[QuantumCircuit]]] = None,
          post_processing: Optional[Callable[[Union[float, List[float]]],
                                             Union[float, List[float]]]] = None) \
        -> LinearSolverResult:
```
The first two are the matrix defining the linear system and the vector right hand side of the equation (i.e. writing $Ax=b$, respectively $A$ and $b$). The remaining parameters concern the (list of) observable(s) to be computed out of the solution vector $x$, and can be specified in two different ways. One option is to give as the third and last parameter a (list of) `LinearSystemObservable`(s). Alternatively, we can give our own implementations of the `observable`, `post_rotation` and `post_processing`, where
- `observable` is the operator to compute the expected value of the observable and can be e.g. a `PauliSumOp`
- `post_rotation` is the circuit to be applied to the solution to extract information if additional gates are needed.
- `post_processing` is the function to compute the value of the observable from the calculated probabilities.

#### Input matrix

There are several ways to specify $A$:
- It can be specified as a numpy array:

```python
import numpy as np
matrix = np.array([[1 / 2, 1 / 6, 0, 0], [1 / 6, 1 / 2, 1 / 6, 0], [0, 1 / 6, 1 / 2, 1 / 6], [0, 0, 1 / 6, 1 / 2]])
```

- If we know how to implement $e^{iAt}$, we can also give this circuit as an input: If the circuit should change for different values of $t$, it should be implemented via the `evolution_time` property (in this case it would be better to create a subclass of `LinearSystemMatrix`):

```python
import scipy as sp
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
evolved = sp.linalg.expm(1j * matrix)
qc.unitary(evolved, qc.qubits)
        
matrix = qc
```

- The matrix can be given as a `LinearSystemMatrix` object:


In [3]:
from qiskit.algorithms.linear_solvers.matrices.tridiagonal_toeplitz import TridiagonalToeplitz
tridi_matrix = TridiagonalToeplitz(2, 1 / 2, 1 / 6)

tridi_solution = HHL().solve(tridi_matrix, vector)

print('classical:', classical_solution.euclidean_norm)
print('tridiagonal matrix:', tridi_solution.euclidean_norm)

classical: 3.98635634469049
tridiagonal matrix: 3.9864914204278157


#### Input vector

HHL also accepts different types for the vector. Again, it can be specified as a numpy array:
```python
vector = np.array([1.0, -2.1, 3.2, -4.3])
```
or as a circuit:


In [4]:
from qiskit import QuantumCircuit

# Initial state circuit
num_qubits = 2
qc = QuantumCircuit(num_qubits)
qc.isometry(vector / np.linalg.norm(vector), list(range(num_qubits)), None)

isom_solution = HHL().solve(matrix, qc)

print('classical:', classical_solution.euclidean_norm)
print('isometry vector:', isom_solution.euclidean_norm)

classical: 3.98635634469049
isometry vector: 3.986685550278159


#### Input observable

Observables are required because the output of the quantum algorithm is a quantum state representing the vector $x$ and learning all the components of this vector would take a linear time in its dimension, diminishing any speedup obtained by the quantum algorithm. Thus, we can only compute functions from $x$ (the so called observables) to learn information about the solution.

There are two types of available `LinearSystemObservable` which can be given as input:

    


In [6]:
from qiskit.algorithms.linear_solvers.observables import AbsoluteAverage, MatrixFunctional

For a vector $\mathbf{x}=(x_1,...,x_N)$, the `AbsoluteAverage` observable computes $|\frac{1}{N}\sum_{i=1}^{N}x_i|$.

In [16]:
average_solution = HHL().solve(matrix, vector, AbsoluteAverage())
classical_average = NumPyLinearSolver().solve(matrix, vector / np.linalg.norm(vector), AbsoluteAverage())

print('quantum average:', average_solution.observable)
print('classical average:', classical_average.observable)
print('quantum circuit results:', average_solution.circuit_results)

quantum average: 0.1783481239084621
classical average: 0.17970391582766393
quantum circuit results: (0.006749775136260498+0j)


The `MatrixFunctional` observable computes $\mathbf{x}^T B \mathbf{x}$ for a vector $\mathbf{x}$ and a tridiagonal symmetric Toeplitz matrix $B$. The class takes the main and off diagonal values of the matrix for its constuctor method.


In [18]:
observable = MatrixFunctional(1, -1 / 3)

functional_solution = HHL().solve(matrix, vector, observable)
classical_functional = NumPyLinearSolver().solve(matrix, vector / np.linalg.norm(vector), observable)

print('quantum functional:', functional_solution.observable)
print('classical functional:', classical_functional.observable)
print('quantum circuit results:', functional_solution.circuit_results)

quantum functional: 24.000239140498067
classical functional: 24.000702987697707
quantum circuit results: [(0.8431720215062039+0j), (0.021698000274993183+0j), (0.8214740212312104+0j), (0.0100715064074334+0j), (0.5004775932428877+0j)]


Finally, the `HHL` class accepts the following parameters in its constructor method:
- error tolerance : the accuracy of the approximation of the solution, the default is `1e-2`
- expectation : how the expectation values are evaluated, the default is `PauliExpectation`
- quantum instance: the `QuantumInstance` or backend, the default is a `Statevector` simulation

In [21]:
from qiskit import BasicAer

backend = BasicAer.get_backend('qasm_simulator')
hhl = HHL(1e-3, quantum_instance=backend)

accurate_solution = hhl.solve(matrix, vector)

print(accurate_solution.euclidean_norm)
print(classical_solution.euclidean_norm)

3.986685550278159
3.98635634469049


C:\Users\AlmudenaCarreraVazqu\Contribute\hhlrefactor\qiskit-terra\qiskit\providers\basicaer\statevector_simulator.py:144: UserWarning: Option backend_options is not used by this backend
  return super().run(qobj, backend_options=backend_options)


### Future work

Negative eigenvalues

In [22]:
matrix = np.array([[1, 2], [2, 1]])
vector = np.array([1, 0])

HHL().solve(matrix, vector)

NotImplementedError: The matrix contains negative eigenvalues, which is not yet supported.